In [2]:
from __future__ import print_function

import sys
import numpy as np
from time import time
from scipy import signal
import matplotlib.pyplot as plt 
import math

sys.path.append('/home/xilinx')
from pynq import Overlay
from pynq import allocate

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))

    print("Start of \"" + sys.argv[0] + "\"")

    ol = Overlay("frft_old.bit")
    ipFRFT = ol.frft_0

    fi_indat = open("input_data.txt", "r+")
    
    numSamples = 256

    inBufferi = allocate(shape=(numSamples,), dtype=np.int32)
    inBufferq = allocate(shape=(numSamples,), dtype=np.int32)    
    outBufferi = allocate(shape=(numSamples,), dtype=np.int32)
    outBufferq = allocate(shape=(numSamples,), dtype=np.int32)
    
    
    line = fi_indat.readline()
    indat = line.split()
    ang = int(math.pi*0.42*(2**14))
    
    
    for i in range(256):
        inBufferi[i] = int(float(indat[i*2])*(2**18))
        inBufferq[i] = int(float(indat[i*2+1])*(2**18))
    fi_indat.close()

    
    #first FrFT
    
    timeKernelStart = time()
    ipFRFT.write(0x30, ang)
    ipFRFT.write(0x20, inBufferi.device_address)
    ipFRFT.write(0x28, inBufferq.device_address)
    ipFRFT.write(0x10, outBufferi.device_address)
    ipFRFT.write(0x18, outBufferq.device_address)
    ipFRFT.write(0x00, 0x01)
    while (ipFRFT.read(0x00) & 0x4) == 0x0:
        continue
    timeKernelEnd = time()
    print("Kernel execution time: " + str(timeKernelEnd - timeKernelStart) + " s")
    
    #Filtering
    
    ang = int(math.pi*-0.42*(2**14))
    for i in range(256):
        if i<(256*0.75) and i>(256*0.25):
            inBufferi[i] = 0
            inBufferq[i] = 0
        else:
            inBufferi[i] = outBufferi[i]
            inBufferq[i] = outBufferq[i]
    
    #Second FrFT
    
    timeKernelStart = time()
    ipFRFT.write(0x30, ang)
    ipFRFT.write(0x00, 0x01)
    while (ipFRFT.read(0x00) & 0x4) == 0x0:
        continue
    timeKernelEnd = time()
    print("Kernel execution time: " + str(timeKernelEnd - timeKernelStart) + " s")


    f = open("filtered.m", "w")
    f.write("x=[...\n")
    for i in range(256):
        f.write("    "+str(float(outBufferi[i])/(2**18))+"+j*"+str(float(outBufferq[i])/(2**18))+", ...\n")
    f.write("];")
    f.close()

    print("============================")
    print("Exit process")



Entry: /usr/lib/python3/dist-packages/ipykernel_launcher.py
System argument(s): 3
Start of "/usr/lib/python3/dist-packages/ipykernel_launcher.py"
Kernel execution time: 0.0037658214569091797 s
Kernel execution time: 0.0039033889770507812 s
Exit process
